Este notebook es para la sistematizacion sin necesariamente usar un diccionario, solo para extraer ideas/palabras principales

In [23]:
import spacy
import pandas as pd
import numpy as np
import string
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import textacy

import funciones as f

In [24]:
nlp = spacy.load('es_core_news_md')

In [92]:
event = 'taller'
df = pd.read_excel('info.xlsx', sheet_name=event)

preguntas = ['P1', 'P2', 'P3']
expanded_responses = {}

In [93]:
def extract_nounphrases(x):
    doc = nlp(x)
    noun_phrases = list(textacy.extract.noun_chunks(doc, min_freq=1))
    return [phrase.text for phrase in noun_phrases]

In [94]:
test_data = {}
for pregunta in preguntas:
    if pregunta in df.columns:
        test_data[pregunta] = df[['Sector', pregunta]].dropna()
        test_data[pregunta]['res'] = test_data[pregunta][pregunta].apply(extract_nounphrases)

In [95]:
file_name = f'res_nodict/{event}.xlsx'
with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    for pregunta, data in test_data.items():
        data.to_excel(writer, sheet_name=pregunta, index=False)

In [ ]:
frequency_data = {}

for pregunta, data in test_data.items():
    all_phrases = [phrase for phrases in data['res'] for phrase in phrases]
    
    phrase_counts = Counter(all_phrases)
    
    frequency_df = pd.DataFrame(phrase_counts.items(), columns=['Phrase', 'Frequency'])
    frequency_df = frequency_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)
    
    frequency_data[pregunta] = frequency_df

In [97]:
file_name = f'res_nodict/resultados_frecuencia_{event}.xlsx'
with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    for pregunta, freq_df in frequency_data.items():
        freq_df.to_excel(writer, sheet_name=f'{pregunta}_freq', index=False)